In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Data Preprocessing
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
onehot_encoder_geography = OneHotEncoder(handle_unknown='ignore')
geography_encoded = onehot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geography_encoded , columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
# Combinrg the encoded geography with the original data
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
# Divide the dataset into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# save the encoder and scaler 
with open('label_encoder_gender.pkl', 'wb') as f:    
    pickle.dump(label_encoder_gender,f)

with open('onehot_encoder_geography.pkl', 'wb') as f:
          pickle.dump(onehot_encoder_geography, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# ANN REGRESSION IMPLEMENTAION

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime


In [12]:
# Build the ANN model
model = Sequential([
          Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected to input layer
          Dense(32, activation='relu'),  ## HL2 connected to HL1
          Dense(1)  ## Output layer

])


# Compile the Model
model.compile(optimizer = 'adam' , loss = "mean_absolute_error" , metrics = ['mae'])

model.summary()

2025-07-02 18:45:52.569647: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-07-02 18:45:52.569909: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-07-02 18:45:52.570490: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-07-02 18:45:52.570611: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-02 18:45:52.571149: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Setup the tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
# setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights =True)

In [15]:
## Train the Model
history = model.fit(
          X_train, y_train, validation_data=(X_test, y_test),
          epochs=100 , 
          callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


2025-07-02 18:47:45.709931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


239/250 [===========================>..] - ETA: 0s - loss: 100462.1406 - mae: 100462.1406

2025-07-02 18:47:48.263241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 3s 5ms/step - loss: 100425.3203 - mae: 100425.3203 - val_loss: 98704.4219 - val_mae: 98704.4219
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 100370.6797 - mae: 100370.6797 - val_loss: 98612.7266 - val_mae: 98612.7266
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 100263.8125 - mae: 100263.8125 - val_loss: 98480.1953 - val_mae: 98480.1953
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 100152.3906 - mae: 100152.3906 - val_loss: 98375.1094 - val_mae: 98375.1094
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 100078.1953 - mae: 100078.1953 - val_loss: 98276.0938 - val_mae: 98276.0938
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 99980.1328 - mae: 99980.1328 - val_loss: 98154.3828 - val_mae: 98154.3828
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 99829.6484 - mae: 99829.6484 - va

In [16]:
# Load tensorboard extension
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit --port 6007


Reusing TensorBoard on port 6007 (pid 22800), started 0:00:09 ago. (Use '!kill 22800' to kill it.)

In [22]:
model.save('regression_model.h5')

/Users/syedmaaz/MAIN/STUDY/Projects/ANN Churn Classification Deep Learning/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
